## Analytical and simulated capped CPW

Analysis of Dunsworth (2018) expression for $p_{ox}$, CR McRae (2017) expression for capped vs uncapped Q, and my own models for loss in a CPW resonator when an oxide is present. Comparison to HFSS simulations to find $p_{ox}$, $p_{ms}$, and $p_{ma}$

In [2]:
from scipy.special import ellipk as K
import numpy as np
from numpy import sinh, tanh
import matplotlib.pyplot as plt
import plot_settings # my personal default settings for matplotlib

_Define elliptic function arguments as found in Simons, Section 2_

In [2]:
def get_k0(S,W, h):
    # conf parameter for isolated CPW, infinite substrate
    # dummy h left in for get_kprime input uniformity
    return S/(S+2*W)

def get_k12(S,W,h):
    # conf parameter for dielectrics below (1) or above(2) CPW
    return sinh(np.pi*S/(4*h))/sinh(np.pi*(S+2*W)/(4*h))

def get_k34(S,W,h):
    # conf parameter for ground plane below (3) or above (4) CPW
    return tanh(np.pi*S/(4*h))/tanh(np.pi*(S+2*W)/(4*h))

def get_kprime(S,W,h,kfunc):
    # passed a k_i parameter, return the respective kprime
    return np.sqrt(1-kfunc(S,W,h)**2)

def G(k, kprime):
    return K(k)/K(kprime)

def get_qi0(S, W, hi):
    # return either q10 or q20 - see N3-132
    return G(S, W, hi, k12)/(2*G(S, W, hi, k0))

def get_q3(S, W, hi, h4):
    # return either q3 or q2 - see N3-132
    
    k0 = get_k0(S, W, None)
    k0prime = get_kprime(S, W, None, get_k0)
    
    ki = get_k12(S, W, hi)
    kiprime = get_kprime(S, W, hi, get_k12)
    
    k4 = get_k34(S, W, h4)
    k4prime = get_kprime(S, W, h4, get_k34)
    
    # Equation 2.40 from Simons
    return G(ki, kiprime)/(G(k0, k0prime) + G(k4, k4prime))
    



_Define functions for analytical quality factor ratios_

In [3]:
def Q_Qcap(S, W, h1, h4, epsr_1):
    """
    Compute the ratio of uncapped CPW to capped CPW - see
    N3-121:132 for derivations etc.
    args:
        S - central conductor width for CPW
        W - CPW gap to ground
        h1 - substrate thickness 
        h4 - cap height
        epsr_1 - substrate relative permittivity
    """
    # ratio of qualities for uncapped/capped resonator quality 
    # h4 is the height of the cap, h1 is the subs
    # CR McRae (2017)
    
    q3 = get_q3(S, W, h1, h4)
    top = (1+epsr_1)*q3
    bot = (epsr_1 - 1)*q3 +1
    return top/bot
    
    return

def Q_Qcap_ox(S, W, h1, h2, h4, epsr_1, epsr_2):
    """
    Compute the ratio of uncapped+oxide CPW to capped+oxide CPE - see
    N3-121:132 for derivations etc.
    args:
        S - central conductor width for CPW
        W - CPW gap to ground
        h1 - substrate thickness 
        h2 - surface "oxide" thickness
        h4 - cap height
        epsr_1 - substrate relative permittivity
        epsr_2 - surface "oxide" relative permittivity
    """
    q1 = get_q3(S, W, h1, h4)
    q2 = get_q3(S, W, h2, h4) # overloading q3 func
    q10 = get_qi0(S, W, h1)
    q20 = get_qi0(S, W, h2)
    
    top = 1 + (epsr_1 -1)*q10 + (epsr_2 -1)*q20 # ~ Q for ox, no cap
    bot = 1 + (epsr_1 -1)*q1 + (epsr_2 -1)*q2 # ~ Q for ox, with cap
    
    
    return top/bot



### Sample analysis 

Plot ratios of Qcpw/Qcap and Q_cpw(ox)/Qcap(ox)

In [ ]:
# CPW parameters, all in um
H1 = 500 # standard substrate
H2 = .01 # 10 nm oxide
H4 = 20 # base height for cap
S = 15
W = 9
EPSR_1 = 10 # Silicon 
EPSR_2 = 10 # standard oxide approx

fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(111)
ax.plot(d_arr, pox_arr)
ax.plot(pox_df.loc[:, "d [um]"], pox_df.loc[:,"F_ox []"])
ax.set_title(r'Analytical vs Simulated $p_{ox}$')
ax.set_xlabel("d [um]")
ax.set_ylabel(r'$p_{ox}$')
plt.grid()
ax.legend(["analytical", "simulated"])
plt.savefig(plot_settings.FIG_DIR+"p_ox")
